# 📈 Stock Quadrant Classification (งบการเงิน + แนวโน้ม)
วิเคราะห์หุ้นและจัดกลุ่มตามระดับงบการเงิน (A–E) และแนวโน้มราคา (a–e) เพื่อสร้าง Quadrant Stock Level Visualization แบบ Plotly

In [ ]:
import pandas as pd
import plotly.graph_objects as go

In [ ]:
# 🔹 ตัวอย่างข้อมูลหุ้น (งบการเงิน + แนวโน้มราคา)
data = {
    'Ticker': ['AOT', 'PTT', 'CPALL', 'SCB', 'BDMS'],
    'ROE': [15.2, 10.5, 12.0, 8.9, 6.0],
    'NetMargin': [18.0, 9.5, 11.2, 7.0, 6.5],
    'DebtEquity': [0.4, 0.6, 1.2, 0.9, 1.1],
    'EPSGrowth': [10, 5, 7, -2, -5],
    'CurrentRatio': [2.0, 1.5, 1.2, 0.8, 0.9],
    'PriceChange': [25, 15, 5, -8, -20]  # % เปลี่ยนแปลงราคาในช่วงที่ผ่านมา
}
df = pd.DataFrame(data)

In [ ]:
# 🔹 ฟังก์ชันจัดกลุ่มหุ้นเป็น Quadrant
def classify_stock_quadrant(df):
    def score_financial(row):
        score = 0
        if row['ROE'] > 12: score += 1
        if row['NetMargin'] > 10: score += 1
        if row['DebtEquity'] < 1: score += 1
        if row['EPSGrowth'] > 0: score += 1
        if row['CurrentRatio'] > 1: score += 1
        return score

    def convert_to_financial_level(score):
        if score == 5: return 'A'
        elif score == 4: return 'B'
        elif score == 3: return 'C'
        elif score == 2: return 'D'
        else: return 'E'

    def trend_level(price_change):
        if price_change > 20: return 'a'
        elif price_change > 10: return 'b'
        elif -5 <= price_change <= 10: return 'c'
        elif -15 < price_change < -5: return 'd'
        else: return 'e'

    df['FinScore'] = df.apply(score_financial, axis=1)
    df['FinancialLevel'] = df['FinScore'].apply(convert_to_financial_level)
    df['TrendLevel'] = df['PriceChange'].apply(trend_level)
    df['Quadrant'] = df['FinancialLevel'] + df['TrendLevel']
    return df[['Ticker', 'FinancialLevel', 'TrendLevel', 'Quadrant']]

In [ ]:
# 🔹 เรียกใช้ฟังก์ชัน
quadrant_df = classify_stock_quadrant(df)
quadrant_df

In [ ]:
# 🔹 แสดงผลในรูปแบบ Plotly Table 5x5

# เตรียม group ข้อมูลตาม Quadrant
quadrant_groups = quadrant_df.groupby('Quadrant')['Ticker'].apply(list).to_dict()

financial_levels = ['A', 'B', 'C', 'D', 'E']
trend_levels = ['a', 'b', 'c', 'd', 'e']

table_text = []
for f in financial_levels:
    row = []
    for t in trend_levels:
        code = f + t
        tickers = quadrant_groups.get(code, [])
        text = f"<b>{code}</b><br>" + "<br>".join(tickers) if tickers else f"<b>{code}</b><br> - "
        row.append(text)
    table_text.append(row)

# สีพื้นหลังตามระดับงบการเงิน
color_map = {
    'A': '#2ecc71', 'B': '#58d68d', 'C': '#f4d03f', 'D': '#f39c12', 'E': '#e74c3c'
}
colors = [[color_map[f]] * 5 for f in financial_levels]

# Plotly Table
fig = go.Figure(data=go.Table(
    header=dict(
        values=["Super Bullish", "Up Trend", "Sideway", "Down Trend", "Crash"],
        align="center",
        fill_color="#dcdcdc",
        font=dict(color="black", size=14)
    ),
    cells=dict(
        values=table_text,
        fill_color=colors,
        align="center",
        height=80,
        font=dict(color="white", size=12)
    )
))
fig.update_layout(title="📊 Quadrant Stock Level (Financial vs Trend)")
fig.show()